# Mapping Police Violence

## Reading Point data

In [ ]:
%matplotlib widget
# %matplotlib notebook


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

PID_Table = pd.read_csv('https://raw.githubusercontent.com/Police-Involved-Deaths-CA/Data/main/Spatial_Data/Geocoded_Points_not_Complete/PID_locations.csv')
PID_Table

# Parse the Data & convert to a Geodataframe

In [ ]:
PID_BC = PID_Table.loc[PID_Table['prov']=='BC']
PID_BC = gpd.GeoDataFrame(
                    PID_BC,
                    geometry=gpd.points_from_xy(PID_BC.longitude, PID_BC.latitude),
                    crs='WGS1984'
                        )

PID_BC.plot()
PID_BC

## Read the Census Data, Re-project the points

To GeoJSON data, we just need to specify the driver.

In [ ]:
BC_FSA = gpd.read_file("data/Outputs/BC_FSA_Clip.json", driver = "GeoJSON")

BC_FSA.crs

PID_BC = PID_BC.to_crs(BC_FSA.crs)

fig,ax=plt.subplots(figsize=(6,6))
BC_FSA.plot(ax=ax)
PID_BC.plot(ax=ax,color='r',edgecolor='k')


ax.set_ylim(PID_BC.geometry.y.min()-5e4,PID_BC.geometry.y.max()+5e4)
ax.set_xlim(PID_BC.geometry.x.min()-5e4,PID_BC.geometry.x.max()+5e4)
ax.set_title('Police Involved Deaths 2016 - Present')

# Vector Overlay


Lets import a points layer for some locations in BC and walk through a handful of vector overlay methods.


## Spatial Joins

We can use a [spatial join](https://geopandas.org/gallery/spatial_joins.html) to merge attributes between two layers based on location

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))

# Changin how to "right" will significantly increase the runtime
# and duplicate each province multiple times (once for each incident within it)
Test_Join = gpd.sjoin(PID_BC, BC_FSA, how="left") 


BC_FSA.plot(ax=ax)
## See if there are any locations "outside" the provincial boundaries
Test_Join.loc[Test_Join['name_right'].isnull()==False].plot(ax=ax,color='k',label='Joined')
Test_Join.loc[Test_Join['name_right'].isnull()].plot(ax=ax,color='r',label='Not Joined')

ax.legend()

Out = Test_Join.loc[Test_Join['name_right'].isnull()]

ax.set_ylim(PID_BC.geometry.y.min()-5e4,PID_BC.geometry.y.max()+5e4)
ax.set_xlim(PID_BC.geometry.x.min()-5e4,PID_BC.geometry.x.max()+5e4)
ax.set_title('Police Involved Deaths 2016 - Present')

print('Not Joined: ',Test_Join.loc[Test_Join['name_right'].isnull()].count()['INDEX'])

# Test_Join.head()
Test_Join.loc[Test_Join['name_right'].isnull(),]

# Try again

* Lets import the "unclipped" FSA file, so we can include the two points that are in the "ocean"

In [ ]:
BC_FSA_Albers = gpd.read_file('data/Outputs/BC_FSA_Albers.shp')

fig,ax=plt.subplots(figsize=(6,6))

# Changin how to "right" will significantly increase the runtime
# and duplicate each province multiple times (once for each incident within it)
Test_Join = gpd.sjoin(PID_BC, BC_FSA_Albers, how="left") 

BC_FSA_Albers.plot(ax=ax)
## See if there are any locations "outside" the provincial boundaries
Test_Join.loc[Test_Join['name_right'].isnull()==False].plot(ax=ax,color='k',label='Joined')
Test_Join.loc[Test_Join['name_right'].isnull()].plot(ax=ax,color='r',label='Not Joined')

ax.legend()

Out = Test_Join.loc[Test_Join['name_right'].isnull()]

ax.set_ylim(PID_BC.geometry.y.min()-5e4,PID_BC.geometry.y.max()+5e4)
ax.set_xlim(PID_BC.geometry.x.min()-5e4,PID_BC.geometry.x.max()+5e4)
ax.set_title('Police Involved Deaths 2016 - Present')

print('Not Joined: ',Test_Join.loc[Test_Join['name_right'].isnull()].count()['INDEX'])

# Test_Join.head()
Test_Join.loc[Test_Join['name_right'].isnull()==False]

## Point In Polygon Analysis

The spatial join method is useful in some cases, but for others, it produces a lot of redundancy.  If your goal is to calculate the number of points per polygon, we can do a point in polygon analysis using the [.within()](https://geopandas.org/docs/reference/api/geopandas.GeoSeries.within.html#geopandas.GeoSeries.within) method. 

* We're going to "cheat" a bit here.  We can use the "unclipped layer" because it's less complicated (fewer vertices).
* It has the same index as the clipped layer, so we can add the points total to the layer we want to display.

In [ ]:
BC_FSA['Deaths'] = 0.0
for i,row in BC_FSA_Albers.iterrows():
#     print(i)
    pip = PID_BC.within(row['geometry'])
#     print(pip)
    if pip.sum()>0:
        BC_FSA.loc[BC_FSA.index==i,'Deaths']+=pip.sum()
print(BC_FSA['Deaths'].describe())

# Masking

* Make a column to differentiate between the FSA with zero incidents

In [ ]:
BC_FSA['Mask']=0
BC_FSA.loc[BC_FSA['Deaths']<1,'Mask']=1

print(BC_FSA.groupby('Mask').count()['spatial_id'])

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
BC_FSA.loc[BC_FSA['Mask']==0].plot(column='Deaths',ax=ax,legend=True,scheme="User_Defined", 
         classification_kwds=dict(bins=[3,6,
             BC_FSA['Deaths'].max()]),
                    edgecolor='k')
BC_FSA.loc[BC_FSA['Mask']==1].plot(ax=ax,color='grey',edgecolor='k')

ax.set_ylim(PID_BC.geometry.y.min()-5e4,PID_BC.geometry.y.max()+5e4)
ax.set_xlim(PID_BC.geometry.x.min()-5e4,PID_BC.geometry.x.max()+5e4)

# Calculating the Death Rate

In [ ]:
BC_FSA['Death_Rate']=BC_FSA['Deaths']/BC_FSA['Population']*1e6/21
BC_FSA['Death_Rate']=BC_FSA['Death_Rate'].fillna(0)

fig,ax=plt.subplots(1,2,figsize = (8,4))

Deaths = BC_FSA.loc[BC_FSA['Mask']==0]

ax[0].scatter(Deaths['Population'],Deaths['Deaths'],edgecolor='k')
ax[0].set_ylabel('Deaths')
ax[0].set_xlabel('Population')


ax[1].scatter(Deaths['Pop_Density'],Deaths['Death_Rate'],edgecolor='k')
ax[1].set_ylabel('Death Rate')
ax[1].set_xlabel('Population')


# Outliers

In [ ]:


fig,ax=plt.subplots(1,2,figsize=(8,4))
BC_FSA['Deaths'].hist(ax=ax[0],bins=30,edgecolor='k')
BC_FSA['Death_Rate'].hist(ax=ax[1],bins=30,edgecolor='k')

print(BC_FSA.loc[BC_FSA['Death_Rate']==BC_FSA['Death_Rate'].max(),
                ['name','Population','Pop_Density','Deaths','Death_Rate']])
print(BC_FSA.loc[BC_FSA['Deaths']==BC_FSA['Deaths'].max(),
                ['name','Population','Pop_Density','Deaths','Death_Rate']])


In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
BC_FSA.loc[BC_FSA['Mask']==0].plot(column='Deaths',ax=ax,legend=True,scheme="User_Defined", 
         classification_kwds=dict(bins=[3,6,
             BC_FSA['Deaths'].max()]),
                    edgecolor='k')
BC_FSA.loc[BC_FSA['Mask']==1].plot(ax=ax,color='grey',edgecolor='k')

PID_BC.plot(ax=ax,facecolor='None'
            ,edgecolor='white')




ax.grid()

fig,ax=plt.subplots(figsize=(6,6))
BC_FSA.loc[BC_FSA['Mask']==0].plot(column='Deaths',ax=ax,legend=True,scheme="User_Defined", 
         classification_kwds=dict(bins=[3,6,
             BC_FSA['Deaths'].max()]),
                    edgecolor='k')
BC_FSA.loc[BC_FSA['Mask']==1].plot(ax=ax,color='grey',edgecolor='k')

PID_BC.plot(ax=ax,facecolor='None'
            ,edgecolor='white')

# ## Set Zoom and turn axes off
x = 1.222e6
y = 0.47e6

Size = 4e4
v = Size/2
h = Size/2

ax.set_xlim(x-h,x+h)
ax.set_ylim(y-h,y+h)


ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)


## Kernel Density

* Kernel Density is a spatial interpolation method, we can use to map the probability of police involved deaths over space

In [ ]:
from sklearn.neighbors import KernelDensity
import pandas as pd
import geopandas as gpd
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import rasterio as rio
from rasterio.plot import show

def kde2D(x, y, bandwidth, cell_size=1e3, **kwargs): 
    """Build 2D kernel density estimate (KDE)."""
    # Transform ipnut points to x,y pairs
    xy_train  = np.vstack([y, x]).T
    # Fit the kernel density model
    kde_skl = KernelDensity(bandwidth=bandwidth, **kwargs)
    kde_skl.fit(xy_train)

    """Construct the Output Image"""
    # Our "Null" hypothesis is a uniform 2D distribution - create a 2D grid
    # Subtract/Add the cell size to the min/max the intervals are fully inclusive of the feature space
    x_ax = np.arange(x.min()-cell_size,x.max()+cell_size,cell_size)
    y_ax = np.arange(y.min()-cell_size,y.max()+cell_size,cell_size)
    xx, yy = np.meshgrid(x_ax,y_ax)
    
    # Transform the grid points to x,y pairs
    xy_test = (np.vstack([yy.ravel(), xx.ravel()]).T)
    
    # score_samples() returns the log-likelihood of the samples
    # convert units to the cell size (z will ~ sum to 1 after conversion)
    z = np.exp(kde_skl.score_samples(xy_test))*cell_size**2
    return xx, yy, np.reshape(z, xx.shape)


x = PID_BC.geometry.x.values
y = PID_BC.geometry.y.values

xx, yy, zz = kde2D(x, y,
                   5000,# 5km band width
                   cell_size=500, #1km cell size
                   kernel='gaussian')

print(zz.min(),zz.max(),zz.mean(),(zz).sum())

fig,ax=plt.subplots(figsize=(10,10))

cb = ax.pcolormesh(xx, yy, zz,shading='auto',cmap='Reds')

BC_FSA.plot(ax=ax,facecolor='None',edgecolor='k')

PID_BC.plot(ax=ax,facecolor='None'
            ,edgecolor='blue')

# ax.legend()
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(cb, cax=cax)


# ## Set Zoom
x = 1.222e6
y = 0.47e6

Size = 4e4
v = Size/2
h = Size/2

ax.set_xlim(x-h,x+h)
ax.set_ylim(y-h,y+h)
ax.grid()

# Save A Raster Image

* We can use Rasterio write a raster layer

In [ ]:

import rasterio as rio
trans = rio.transform.from_bounds(xx.min(), yy.min(), xx.max(), yy.max(), int(xx.shape[0]), int(yy.shape[1]))

print(xx.min(), yy.min(), xx.max(), yy.max())
print(trans)

with rio.open('Kernel_Density.tif', 'w',
                  dtype=rio.float32,
                  count=1,
                  compress='lzw',
                  width=int(xx.shape[0]),
                  height=int(yy.shape[1]),
                  transform=trans) as dst:
    dst.write(np.flip(zz,axis=0).astype(rio.float32), 1)
print('Saved')

# Plot the Raster

In [ ]:

from rasterio.plot import show
fig,ax=plt.subplots()
with rio.open('Kernel_Density.tif','r') as Test:
    show(Test,ax=ax,cmap='Reds')
    
BC_FSA.plot(ax=ax,facecolor='None',edgecolor='k')
    
    
# ## Set Zoom & Turn Grid off
x = 1.222e6
y = 0.47e6

Size = 4e4
v = Size/2
h = Size/2